In [170]:
import pandas as pd
import plotly.express as px
import numpy as np
import os


In [171]:
df = pd.read_csv('csv_exports/op_whale_actions_22_01_17.csv')
df['usd_value_in'] = np.where(df['token_direction'] == 'in',df['usd_value'], 0)
df['usd_value_out'] = np.where(df['token_direction'] == 'out',df['usd_value'], 0)
df['usd_value_net'] = df['usd_value_in'] - df['usd_value_out']
df['usd_value_net'] = np.where(df['usd_value_net'] < 0, 0, df['usd_value_net'])
val_cols = ['usd_value','usd_value_in','usd_value_out','usd_value_net']

In [172]:
df_addr = df.groupby(['address','project','contract','method']).sum()
df_addr.reset_index(inplace=True)

for val in val_cols:
        df_addr[val + '_pct_share'] = df_addr.groupby('address')[val].transform(lambda x: x / x.sum())
# display(df_addr)

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_88644/3317580168.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [173]:
df_dollar = df[['project','contract','method','usd_value','usd_value_in','usd_value_out','usd_value_net','address']]
df_dollar = df_dollar.groupby(['project','contract','method']).agg({
        'usd_value':'sum'
        ,'usd_value_in':'sum'
        ,'usd_value_out':'sum'
        ,'usd_value_net':'sum'
        ,'address':'nunique'
})
df_dollar.reset_index(inplace=True)
df_dollar.name = 'df_dollar'
for val in val_cols:
        df_dollar[val + '_pct_share'] =  df_dollar[val] / ( df_dollar[val].sum() )


In [174]:
# Get the average volume share across all whales
# i.e. once someone is determined to be a whale, tkae their pct transfer share, and then we avg across rather than $ weight.
# The reason to not dollar-weight is that we care more about genetating the average individual user experience, rather than the whaliest whale.

df_action = df_addr.groupby(['project','contract','method']).agg({
        'usd_value_pct_share':'mean'
        ,'usd_value_in_pct_share':'mean'
        ,'usd_value_out_pct_share':'mean'
        ,'usd_value_net_pct_share':'mean'
        ,'address':'nunique'
})
df_action.name = 'df_action'

# display(df_action)

In [175]:
dfs = [df_action, df_dollar]
dlist = ['project','contract','method','action',\
                'usd_value_pct_share','usd_value_in_pct_share','usd_value_out_pct_share','usd_value_net_pct_share','address']


if not os.path.exists("csv_outputs"):
        os.mkdir("csv_outputs")
        
for i, d in enumerate(dfs):
        original_name = d.name
        d.reset_index(inplace=True)
        
        d['action'] = d['project'].astype(str) + ' - ' + d['contract'].astype(str) + ' | ' + d['method'].astype(str)
        d = d[dlist]
        
        dfs[i] = d
        d.name = original_name
        
#dumb hardcode, tried to do it fancy, but whatever
df_action = dfs[0]
df_dollar = dfs[1]

df_action.to_csv('csv_outputs/whale_actions_share.csv')
df_dollar.to_csv('csv_outputs/whale_actions_share_dollar_weight.csv')
# gix = px.pie(df_action, values='usd_value_out_pct_share', names='action', title='Share of Actions')